In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_newww.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [5]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [6]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [8]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [9]:
# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(50, name='lstm_layer'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax', name='output_layer'))

In [10]:
# Compile the model initially
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Iterative layer freezing
for i in range(len(lstm_model.layers)):
    # Freeze all layers up to the current layer
    for j in range(i + 1):
        lstm_model.layers[j].trainable = False

    # Re-compile the model to apply changes
    lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print(f"Training with layers 0 to {i} frozen...")

    # Train model for a few epochs
    lstm_model.fit(X_train_lstm, y_train, epochs=5, batch_size=32, validation_data=(X_test_lstm, y_test))

    # Evaluate the model on the validation set
    val_loss, val_accuracy = lstm_model.evaluate(X_test_lstm, y_test)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

Training with layers 0 to 0 frozen...
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2848 - loss: 1.6522 - val_accuracy: 0.3204 - val_loss: 1.5694
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3042 - loss: 1.5402 - val_accuracy: 0.4807 - val_loss: 1.4770
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3908 - loss: 1.4703 - val_accuracy: 0.5912 - val_loss: 1.4001
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4809 - loss: 1.3998 - val_accuracy: 0.6354 - val_loss: 1.3367
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5404 - loss: 1.3408 - val_accuracy: 0.6188 - val_loss: 1.2830
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6199 - loss: 1.2995 
Validation Loss: 1.2830450534820557, Validation Accuracy: 0.6187845468521118
Training with layers 0 to 1 frozen...
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5407 - loss: 1.2911 - val_accuracy: 0.6796 - val_loss: 1.2302
Epoch 2/5
23/23 ━━━━━━━

C:\Users\HP\anaconda3\envs\mediapipe\lib\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6592 - loss: 1.1268 - val_accuracy: 0.7293 - val_loss: 1.0751
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6355 - loss: 1.1280 - val_accuracy: 0.7293 - val_loss: 1.0751
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6531 - loss: 1.1288 - val_accuracy: 0.7293 - val_loss: 1.0751
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6668 - loss: 1.1037 - val_accuracy: 0.7293 - val_loss: 1.0751
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6685 - loss: 1.1187 - val_accuracy: 0.7293 - val_loss: 1.0751
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7049 - loss: 1.0992
Validation Loss: 1.0751192569732666, Validation Accuracy: 0.7292817831039429


In [12]:
# After loop, unfreeze all layers for final fine-tuning
for layer in lstm_model.layers:
    layer.trainable = True

In [13]:
# Compile the model again for fine-tuning
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Perform fine-tuning
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7288 - loss: 0.9939 - val_accuracy: 0.8950 - val_loss: 0.7901
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8039 - loss: 0.8059 - val_accuracy: 0.8895 - val_loss: 0.6782
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8152 - loss: 0.7314 - val_accuracy: 0.8674 - val_loss: 0.6246
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8505 - loss: 0.6622 - val_accuracy: 0.9006 - val_loss: 0.5438
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8576 - loss: 0.5864 - val_accuracy: 0.9006 - val_loss: 0.4993
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8833 - loss: 0.5582 - val_accuracy: 0.9116 - val_loss: 0.4715
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8764 - loss: 0.5186 - val_accuracy: 0.9227 - val_loss: 0.4356
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8749 - loss: 0.4988 - val_accuracy: 0.9171 - val_loss

In [15]:
# Save the training history
with open('training_history_newdataset.json', 'w') as f:
    json.dump(lstm_history.history, f)

In [16]:
# Save the final LSTM model
lstm_model.save('verification_model_newdataset.h5')